<a href="https://colab.research.google.com/github/siderbrand/PROYECTO-KAGGLE/blob/main/03_modelo_con_preprocesado_TE_%2B_Ordinal_%2B_Scaling_y_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center"><img alt="udeA logo" height="150px" src="https://github.com/freddyduitama/images/blob/master/logo.png?raw=true"></p><h1><font color='0B5345'> <center>

</center></font></h1>
<center>
INTRODUCCION A LA INTELIGENIA ARTIFICIAL

 </center></font></h1>
<h2><font color='0B5345'> <center>
PROYECTO KAGGLE</center></font></h2>
<font  face="Courier New" size="3">

<p3><center><b><font color='0B5345' face="Lucida Calligraphy,Comic Sans MS,Lucida Console" size="5">Universidad de Antioquia</font></b> </center></p3>

<p3><center><b><font> Juan Andres Toro Acevedo - Bioingenieria
<p3><center><b><font> Eliana María Brand Agudelo - Ingenieria en Sistemas
<p3><center><b><font> Claudia María Rocha Hernández- Ingenieria en Sistemas

Instalacion de librerias

In [ ]:
!pip install -q scikit-learn pandas joblib tqdm lightgbm


Montar en Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


Cargar la ruta para el archivo preprocesado y el path para cargar el output

In [ ]:

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/Introduccion IA/cleaned_data.csv"
OUTPUT_MODEL = "/content/drive/MyDrive/Colab Notebooks/Introduccion IA/modelo_svm_colab.joblib"


Imports necesarios

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd

from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegression

from lightgbm import LGBMClassifier

from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


Target Encoding K-Fold


In [ ]:
def target_encode_kfold_train(X_train, y_train, cols, n_splits=5, random_state=42):
    X_train = X_train.copy()
    mappings = {}
    if len(cols) == 0:
        return X_train, mappings

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for c in cols:
        vals = X_train[c].astype(str).fillna('')
        oof = pd.Series(index=X_train.index, dtype=float)

        codes, uniques = pd.factorize(pd.Series(y_train).reset_index(drop=True))
        y_arr = np.array(codes)
        prior = float(np.mean(y_arr))

        for train_idx, val_idx in cv.split(X_train, y_train):
            train_vals = vals.iloc[train_idx]
            train_y = y_arr[train_idx]

            stats = pd.DataFrame({
                'val': train_vals,
                'y': train_y
            }).groupby('val')['y'].agg(['mean', 'count'])

            k = 5.0
            smoothed = ((stats['mean'] * stats['count']) + prior * k) / (stats['count'] + k)
            mapped = vals.iloc[val_idx].map(smoothed).fillna(prior)
            oof.iloc[val_idx] = mapped

        full_stats = pd.DataFrame({
            'val': vals,
            'y': y_arr
        }).groupby('val')['y'].agg(['mean', 'count'])

        k = 5.0
        full_smoothed = ((full_stats['mean'] * full_stats['count']) +
                         prior * k) / (full_stats['count'] + k)

        mappings[c] = {
            'mapping': full_smoothed.to_dict(),
            'global_mean': float(prior),
            'label_map': {int(i): v for i, v in enumerate(uniques)}
        }

        X_train[c + '_te'] = oof.fillna(prior)

    return X_train, mappings


Funcion para aplicar target encoding

In [ ]:
def apply_te_to_df(X, mappings):
    X = X.copy()
    for c, info in mappings.items():
        if c not in X.columns:
            continue
        mapping = {str(k): float(v) for k, v in info['mapping'].items()}
        gmean = info['global_mean']

        X[c + '_te'] = X[c].astype(str).map(mapping).fillna(gmean).astype(float)

    return X


Construccion del Pipeline

In [ ]:
def build_pipeline(X_sample, model_choice='sgd'):
    numeric_cols = X_sample.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X_sample.select_dtypes(exclude=[np.number]).columns.tolist()

    if len(categorical_cols) > 0:
        X_sample[categorical_cols] = X_sample[categorical_cols].astype(str).fillna('')

    num_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median'))
    ])

    ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    cat_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='')),
        ('ord', ord_enc)
    ])

    preproc = ColumnTransformer([
        ('num', num_pipe, numeric_cols),
        ('cat', cat_pipe, categorical_cols)
    ], remainder='drop')

    model_choice = model_choice.lower()

    if model_choice == 'sgd':
        base = SGDClassifier(max_iter=1000, tol=1e-3, class_weight='balanced')
        clf = CalibratedClassifierCV(base, cv=3)
        pipe = Pipeline([
            ('preproc', preproc),
            ('scaler', StandardScaler()),
            ('clf', clf)
        ])

    elif model_choice == 'logreg':
        base = LogisticRegression(
            solver='saga', multi_class='multinomial',
            max_iter=1000, class_weight='balanced'
        )
        pipe = Pipeline([
            ('preproc', preproc),
            ('scaler', StandardScaler()),
            ('clf', base)
        ])

    elif model_choice == 'lgbm':
        base = LGBMClassifier(objective='multiclass')
        pipe = Pipeline([
            ('preproc', preproc),
            ('clf', base)
        ])

    else:
        raise ValueError("Modelo no soportado:", model_choice)

    return pipe


In [ ]:
TARGET = "RENDIMIENTO_GLOBAL"
USE_TE = True
MODEL = "sgd"                     # sgd | logreg | lgbm
N_ITER = 30
N_JOBS = 2
TEST_SIZE = 0.2
TE_COLS = 5
RANDOM_STATE = 42


Cargar dataset y preparar el target encoding

In [ ]:
df = pd.read_csv(DATA_PATH)
print("Dataset cargado:", df.shape)

df = df.dropna(subset=[TARGET])
X = df.drop(columns=[TARGET])
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_STATE
)

print("Train:", X_train.shape, "Test:", X_test.shape)

te_mappings = {}

if USE_TE:
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()
    cat_cols = sorted(cat_cols, key=lambda c: X_train[c].nunique(), reverse=True)
    te_cols = [c for c in cat_cols if X_train[c].nunique() > 3][:TE_COLS]

    print("TE aplicado en:", te_cols)

    X_train, te_mappings = target_encode_kfold_train(X_train, y_train, te_cols)
    X_test = apply_te_to_df(X_test, te_mappings)

    # Frequency Encoding
    for c in cat_cols:
        freq_map = X_train[c].astype(str).value_counts(normalize=True).to_dict()
        X_train[c + "_freq"] = X_train[c].astype(str).map(freq_map).fillna(0.0)
        X_test[c + "_freq"] = X_test[c].astype(str).map(freq_map).fillna(0.0)


Dataset cargado: (692500, 19)
Train: (554000, 18) Test: (138500, 18)
TE aplicado en: ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO']


Construccion del pipelinee y RandomizedSearchCV

In [ ]:
pipeline = build_pipeline(X_train.head(1000), model_choice=MODEL)

if MODEL == 'sgd':
    param_dist = {
        'clf__estimator__loss': ['hinge', 'log_loss'],
        'clf__estimator__penalty': ['l2', 'l1', 'elasticnet'],
        'clf__estimator__alpha': [1e-4, 1e-3, 1e-2],
        'clf__estimator__eta0': [0.0, 1e-3],
    }

elif MODEL == 'logreg':
    param_dist = {
        'clf__C': [0.01, 0.1, 1.0, 5.0],
        'clf__penalty': ['l2', 'elasticnet'],
        'clf__l1_ratio': [0.0, 0.15, 0.5],
    }

elif MODEL == 'lgbm':
    param_dist = {
        'clf__n_estimators': [100, 300, 600],
        'clf__learning_rate': [0.01, 0.05, 0.1],
        'clf__max_depth': [4, 6, 8],
        'clf__num_leaves': [31, 50, 100],
        'clf__subsample': [0.7, 0.8, 1.0],
        'clf__colsample_bytree': [0.7, 0.8, 1.0],
    }

rs = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=N_ITER,
    cv=3,
    scoring='accuracy',
    n_jobs=N_JOBS,
    random_state=RANDOM_STATE,
    verbose=1
)

print("Iniciando RandomizedSearchCV...")
rs.fit(X_train, y_train)

print("Best params:", rs.best_params_)
print("Best CV score:", rs.best_score_)


/tmp/ipython-input-3860950996.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_sample[categorical_cols] = X_sample[categorical_cols].astype(str).fillna('')


Iniciando RandomizedSearchCV...
Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best params: {'clf__estimator__penalty': 'l2', 'clf__estimator__loss': 'log_loss', 'clf__estimator__eta0': 0.001, 'clf__estimator__alpha': 0.001}
Best CV score: 0.4044169659835524


Evaluacion del mejor modelo

In [ ]:
best = rs.best_estimator_

y_pred = best.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy Test:", acc)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy Test: 0.40306137184115526

Classification Report:
              precision    recall  f1-score   support

        alto       0.46      0.68      0.55     35124
        bajo       0.42      0.57      0.48     34597
  medio-alto       0.30      0.15      0.20     34324
  medio-bajo       0.32      0.21      0.25     34455

    accuracy                           0.40    138500
   macro avg       0.37      0.40      0.37    138500
weighted avg       0.37      0.40      0.37    138500


Confusion Matrix:
[[23838  4095  3859  3332]
 [ 5532 19844  3378  5843]
 [13890  9387  4990  6057]
 [ 8810 13941  4552  7152]]


Guardar el modelo en drive en el path definido para el output

In [ ]:
joblib.dump(
    {
        'model': best,
        'te_mappings': te_mappings,
        'target': TARGET
    },
    OUTPUT_MODEL
)

print("Modelo guardado en:", OUTPUT_MODEL)


Modelo guardado en: /content/drive/MyDrive/Colab Notebooks/Introduccion IA/modelo_svm_colab.joblib
